In [ ]:
pip install researchpy

In [ ]:
import researchpy as rp
import pandas as pd
import numpy as np
from scipy import stats
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd


##TO MODIFY BY USER
### Instruction: You will need to run thise codeonce for each Assessment number wanted (A0, A1, A2) 
#### Enter the directory with the cleaned csv file (folder with the file generated by 1_DataCleaner)
path = '/Users/malika/Documents/Five Lives/SI Data Analysis/LifestyleCognition/Datasets & Data cleaning/'
Data = pd.read_csv(path + 'DataframeCogPerfA0.csv')

plot = '/Users/malika/Documents/Five Lives/SI Data Analysis/LifestyleCognition/HierarchicalClusteringAnalysis/A0-RT/Plots - Regression and clustering/'

Clustering Analyisis
Great tutorial: https://stackabuse.com/hierarchical-clustering-with-python-and-scikit-learn/

In [ ]:
Data = Data.set_index('participant_id')

In [ ]:
Data.columns

In [ ]:
lifestyle = ['A0PA_duration',
 'A0PA_intensity',
 'A0PA_score',
 'A0smoking_situation',
 'A0smoking_amount',
 'A0socialI',
 'A0loneliness',
 'A0sleep_problems',
 'A0fruit_veg',
 'A0alcohol_unit',
 'A0glasses_water',
 'A0time_reading',
 'A0time_mental_stimulation',
 'A0time_instrument',
 'A0time_skill',
 'A0relaxed_scale',
 'A0mood_rate',
 'A0enjoyment_activities']

In [ ]:
cog = ['A0cast_performance',
       'A0snap_category_performance', 'A0snap_initials_performance',
       'A0swift_incongruent_performance', 'A0swift_congruent_performance',
       'A0twist_performance']

In [ ]:
Data

## Lifestyle factors predicting cognition

In [ ]:
## A0PA_duration predicts A0snap_category_performance

In [ ]:
c = 'A0_age + A0male'
for to_predict in cog:
    # drop nan in each subset to predict to avoid errors due to too many nan
    model_sum = ['by ' + to_predict]
    for predictor in lifestyle:
        model_sum.append(predictor)
        df = Data.dropna(axis = 0, subset = [to_predict, predictor])
        try:
            #Model
            m = to_predict + ' ~ ' + predictor + ' + ' + c
            mdf_p = OLS.from_formula(m, df)
            mdf = mdf_p.fit()
            #Create a list to save the models
            model_sum.append(mdf.summary())
            file_name = to_predict
            path = plot + file_name+'.txt'
            fp= open(path, 'w')
            with open(path, 'w') as fp:
                for item in model_sum:
                    # write each item on a new line
                    fp.write("%s\n" % item)
                    print('Done')
        except:
            fail = to_predict + '_by_' + predictor
            fail_sum.append(fail)
            path = plot + 'ErrorRaised.txt'
            fp= open(path, 'w')
            with open(path, 'w') as fp:
                for item in fail_sum:
                    # write each item on a new line
                    fp.write("%s\n" % item)
                    print('Done')

In [ ]:
physical_var = ['A0PA_duration', 'A0PA_intensity', 'A0PA_score']
sleep_var = 'A0sleep_problems'
diet_var =  ['A0fruit_veg', 'A0alcohol_unit', 'A0glasses_water']

smoking_var =  ['A0smoking_situation', 'A0smoking_amount']
mood_var =  ['A0relaxed_scale', 'A0mood_rate', 'A0enjoyment_activities']
mental_stim_var =  ['A0socialI', 'A0loneliness', 'A0time_mental_stimulation', 'A0time_instrument', 'A0time_skill', 'A0time_reading']


In [ ]:
FL_pillars = [physical_var, sleep_var, diet_var, mood_var, mental_stim_var, smoking_var ]

In [ ]:
for predictor in FL_pillars:
    for i in range(1, len(predictor)):
        m_predictor = predictor[0]
        m_predictor = m_predictor + ' + ' + predictor[i]

In [ ]:
# Repeat models but on different pillars

smoking =  'A0smoking_situation + A0smoking_amount'
FL_pillars = []

## Model reproeduced on FL pillars
FL_pillars = [physical_var, sleep_var, diet_var, mood_var, mental_stim_var, smoking_var ]
fail_sum = []
### For list for cognitive change variables
for to_predict in cog:
    df = Data.dropna(axis = 0, subset = [to_predict])
    # drop nan in each subset to predict to avoid errors due to too many nan
    model_sum = [to_predict]
    for predictor in FL_pillars:
        df = Data.dropna(axis = 0, subset = predictor)
        try:
            #Model
            ## Build the model formula based on the listt of predictors
            for i in range(1, len(predictor)):
                m_predictor = predictor[0]
                m_predictor = m_predictor + ' + ' + predictor[i]
            
            m = to_predict + ' ~ ' + m_predictor + ' + ' + c
            mdf_p = OLS.from_formula(m, df)
            mdf = mdf_p.fit()
            #Create a list to save the models
            model_sum.append(mdf.summary())
            file_name = to_predict
            path = plot + 'pillar/'+ file_name+'.txt'
            fp= open(path, 'w')
            with open(path, 'w') as fp:
                for item in model_sum:
                    # write each item on a new line
                    fp.write("%s\n" % item)
                    print('Done')
        except:
            fail = to_predict + '_by_' + m_predictor
            fail_sum.append(fail)
            path = plot + 'pillar/'+ file_name+'.txt'
            fp= open(path, 'w')
            with open(path, 'w') as fp:
                for item in fail_sum:
                    # write each item on a new line
                    fp.write("%s\n" % item)
                    print('Done')

most significant results are found for snap category
among the LS variables significant, the 3 that stood out as most significant are time_reading',
time_mental_stimulation',
relaxed_scale',

Lets explore with those

# Hierarchical Clustering
## 2 variables

time_reading & time_mental_stimulation

In [ ]:
df = Data[['A0time_reading', 'A0time_mental_stimulation' ]].dropna(axis=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
X = df.values
X= StandardScaler().fit_transform(X) # normalizing the features

In [ ]:
np.mean(X), np.std(X)

In [ ]:
#3 Using the dendrogram to find the optimal numbers of clusters. 
# First thing we're going to do is to import scipy library. scipy is #an open source Python library that contains tools to do #hierarchical clustering and building dendrograms. Only import the #needed tool.
import scipy.cluster.hierarchy as sch
#Lets create a dendrogram variable linkage is actually the algorithm #itself of hierarchical clustering and then in linkage we have to #specify on which data we apply and engage. This is X dataset
dendrogram = sch.dendrogram(sch.linkage(X, method  = "ward"))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.show()

In [ ]:
#4 Fitting hierarchical clustering to the Mall_Customes dataset
# There are two algorithms for hierarchical clustering: #Agglomerative Hierarchical Clustering and 
# Divisive Hierarchical Clustering. We choose Euclidean distance and ward method for our algorithm class
from sklearn.cluster import AgglomerativeClustering 
hc = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage ='ward')
# Lets try to fit the hierarchical clustering algorithm  to dataset #X while creating the clusters vector that tells for each customer #which cluster the customer belongs to.
y_hc=hc.fit_predict(X)

In [ ]:
y_hc

In [ ]:
len(X)

In [ ]:
#5 Visualizing the clusters. This code is similar to k-means #visualization code. We only replace the y_kmeans vector name to #y_hc for the hierarchical clustering
plt.scatter(X[y_hc==0, 0], X[y_hc==0, 1], s=100, c='red', label ='Cluster 1')
plt.scatter(X[y_hc==1, 0], X[y_hc==1, 1], s=100, c='blue', label ='Cluster 2')
plt.scatter(X[y_hc==2, 0], X[y_hc==2, 1], s=100, c='green', label ='Cluster 3')
#plt.scatter(X[y_hc==3, 0], X[y_hc==3, 1], s=100, c='cyan', label ='Cluster 4')
#plt.scatter(X[y_hc==4, 0], X[y_hc==4, 1], s=100, c='magenta', label ='Cluster 5')
plt.title('Clusters of Participants (Hierarchical Clustering Model)')
plt.xlabel('A0time_reading')
plt.ylabel('A0time_mental_stimulation')
plt.show()

In [ ]:
len(y_hc)

In [ ]:
len(df)

In [ ]:
df['cluster_label'] = y_hc
dfa = df.join(Data['A0snap_category_performance'])

In [ ]:

dfa.isnull().sum()

In [ ]:
dfa.dropna(axis=0)

In [ ]:
rp.summary_cont(dfa['A0snap_category_performance'].groupby(dfa['cluster_label']))

In [ ]:
title = dfa.columns[0] + ' & ' + dfa.columns[1]

In [ ]:
title

In [ ]:
title = dfa.columns[0] + ' & ' + dfa.columns[1]
x ='A0snap_category_performance'
    #define mean and median
median = dfa[x].median()
mean = "%.2f" % dfa[x].mean() 
    #plot figures
plt.figure(figsize=(3, 5))
sns.displot(data = dfa, x = x, hue = 'cluster_label')
plt.axvline(dfa[x].median(), linestyle='dotted', color = 'magenta', label = f'median= {median}')
plt.axvline(dfa[x].mean(), linestyle='dotted', color = 'blue', label = f'mean= {mean}')


plt.legend(loc = 'upper right')
plt.title(title + ':A0snap_category_performance by group cluster')

In [ ]:
plt.figure(figsize=(3, 5))
sns.boxplot(data = dfa, x = 'cluster_label', y = 'A0snap_category_performance', color = 'pink')
plt.title(title + ':A0snap_category_performance by group cluster')

In [ ]:
dfb = dfa.dropna(axis=0)

In [ ]:
dfb

In [ ]:
dfb.groupby('cluster_label')['A0snap_category_performance']

In [ ]:
dfb

In [ ]:
dfb.groupby('cluster_label')

In [ ]:
grpe0 = dfb['A0snap_category_performance'][dfb['cluster_label']==0]
grpe1 = dfb['A0snap_category_performance'][dfb['cluster_label']==1]
grpe2 = dfb['A0snap_category_performance'][dfb['cluster_label']==2]


In [ ]:
f_val, p_val = stats.f_oneway(grpe0, grpe1, grpe2)  
 
print( "ANOVA results: F=", f_val, ", P =", p_val)  


### time reading and relaxed scale

In [ ]:
df = Data[['A0time_reading', 'A0relaxed_scale' ]].dropna(axis=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
X = df.values
X= StandardScaler().fit_transform(X) # normalizing the features

In [ ]:
np.mean(X), np.std(X)

In [ ]:
#3 Using the dendrogram to find the optimal numbers of clusters. 
# First thing we're going to do is to import scipy library. scipy is #an open source Python library that contains tools to do #hierarchical clustering and building dendrograms. Only import the #needed tool.
import scipy.cluster.hierarchy as sch
#Lets create a dendrogram variable linkage is actually the algorithm #itself of hierarchical clustering and then in linkage we have to #specify on which data we apply and engage. This is X dataset
dendrogram = sch.dendrogram(sch.linkage(X, method  = "ward"))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.show()

In [ ]:
#4 Fitting hierarchical clustering to the Mall_Customes dataset
# There are two algorithms for hierarchical clustering: #Agglomerative Hierarchical Clustering and 
# Divisive Hierarchical Clustering. We choose Euclidean distance and ward method for our algorithm class
from sklearn.cluster import AgglomerativeClustering 
hc = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage ='ward')
# Lets try to fit the hierarchical clustering algorithm  to dataset #X while creating the clusters vector that tells for each customer #which cluster the customer belongs to.
y_hc=hc.fit_predict(X)

#5 Visualizing the clusters. This code is similar to k-means #visualization code. We only replace the y_kmeans vector name to #y_hc for the hierarchical clustering
plt.scatter(X[y_hc==0, 0], X[y_hc==0, 1], s=100, c='red', label ='Cluster 1')
plt.scatter(X[y_hc==1, 0], X[y_hc==1, 1], s=100, c='blue', label ='Cluster 2')
plt.scatter(X[y_hc==2, 0], X[y_hc==2, 1], s=100, c='green', label ='Cluster 3')
#plt.scatter(X[y_hc==3, 0], X[y_hc==3, 1], s=100, c='cyan', label ='Cluster 4')
#plt.scatter(X[y_hc==4, 0], X[y_hc==4, 1], s=100, c='magenta', label ='Cluster 5')
plt.title('Clusters of Participants (Hierarchical Clustering Model)')
plt.xlabel('A0time_reading')
plt.ylabel('A0relaxed_scale')
plt.show()


In [ ]:
df['cluster_label'] = y_hc
dfa = df.join(Data['A0snap_category_performance'])
dfa = dfa.dropna(axis=0)

In [ ]:
dfa

In [ ]:
title = dfa.columns[0] + ' & ' + dfa.columns[1]

x ='A0snap_category_performance'
    #define mean and median
median = dfa[x].median()
mean = "%.2f" % dfa[x].mean() 
    #plot figures
plt.figure(figsize=(3, 5))
sns.displot(data = dfa, x = x, hue = 'cluster_label')
plt.axvline(dfa[x].median(), linestyle='dotted', color = 'magenta', label = f'median= {median}')
plt.axvline(dfa[x].mean(), linestyle='dotted', color = 'blue', label = f'mean= {mean}')


plt.legend(loc = 'upper right')
plt.title(title + ': A0snap_category_performance by group cluster')

In [ ]:
plt.figure(figsize=(3, 5))
sns.boxplot(data = dfa, x = 'cluster_label', y = 'A0snap_category_performance', color = 'pink')
plt.title('Time reading & Relaxed scale:A0snap_category_performance by group cluster')

In [ ]:
import pandas as pd
rp.summary_cont(dfa['cluster_label'])


In [ ]:
rp.summary_cont(dfa['A0snap_category_performance'].groupby(dfa['cluster_label']))


In [ ]:
grpe0 = dfa['A0snap_category_performance'][dfa['cluster_label']==0]
grpe1 = dfa['A0snap_category_performance'][dfa['cluster_label']==1]
grpe2 = dfa['A0snap_category_performance'][dfa['cluster_label']==2]

f_val, p_val = stats.f_oneway(grpe0, grpe1, grpe2)  
 
print( "ANOVA results: F=", f_val, ", P =", p_val)  

### time mental stim and relaxed scale

In [ ]:
df = Data[['A0time_mental_stimulation', 'A0relaxed_scale' ]].dropna(axis=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
X = df.values
X= StandardScaler().fit_transform(X) # normalizing the features

In [ ]:
np.mean(X), np.std(X)

In [ ]:
#3 Using the dendrogram to find the optimal numbers of clusters. 
# First thing we're going to do is to import scipy library. scipy is #an open source Python library that contains tools to do #hierarchical clustering and building dendrograms. Only import the #needed tool.
import scipy.cluster.hierarchy as sch
#Lets create a dendrogram variable linkage is actually the algorithm #itself of hierarchical clustering and then in linkage we have to #specify on which data we apply and engage. This is X dataset
dendrogram = sch.dendrogram(sch.linkage(X, method  = "ward"))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.show()

In [ ]:
#4 Fitting hierarchical clustering to the Mall_Customes dataset
# There are two algorithms for hierarchical clustering: #Agglomerative Hierarchical Clustering and 
# Divisive Hierarchical Clustering. We choose Euclidean distance and ward method for our algorithm class
from sklearn.cluster import AgglomerativeClustering 
hc = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage ='ward')
# Lets try to fit the hierarchical clustering algorithm  to dataset #X while creating the clusters vector that tells for each customer #which cluster the customer belongs to.
y_hc=hc.fit_predict(X)

#5 Visualizing the clusters. This code is similar to k-means #visualization code. We only replace the y_kmeans vector name to #y_hc for the hierarchical clustering
plt.scatter(X[y_hc==0, 0], X[y_hc==0, 1], s=100, c='red', label ='Cluster 1')
plt.scatter(X[y_hc==1, 0], X[y_hc==1, 1], s=100, c='blue', label ='Cluster 2')
plt.scatter(X[y_hc==2, 0], X[y_hc==2, 1], s=100, c='green', label ='Cluster 3')
#plt.scatter(X[y_hc==3, 0], X[y_hc==3, 1], s=100, c='cyan', label ='Cluster 4')
#plt.scatter(X[y_hc==4, 0], X[y_hc==4, 1], s=100, c='magenta', label ='Cluster 5')
plt.title('Clusters of Participants (Hierarchical Clustering Model)')
plt.xlabel('A0time_mental_stimulation')
plt.ylabel('A0relaxed_scale')
plt.show()

In [ ]:
df['cluster_label'] = y_hc
dfa = df.join(Data['A0snap_category_performance'])
dfa = dfa.dropna(axis=0)

In [ ]:
dfa

In [ ]:
title = dfa.columns[0] + ' & ' + dfa.columns[1]

x ='A0snap_category_performance'
    #define mean and median
median = dfa[x].median()
mean = "%.2f" % dfa[x].mean() 
    #plot figures
plt.figure(figsize=(3, 5))
sns.displot(data = dfa, x = x, hue = 'cluster_label')
plt.axvline(dfa[x].median(), linestyle='dotted', color = 'magenta', label = f'median= {median}')
plt.axvline(dfa[x].mean(), linestyle='dotted', color = 'blue', label = f'mean= {mean}')


plt.legend(loc = 'upper right')
plt.title(title + ': A0snap_category_performance by group cluster')

In [ ]:
plt.figure(figsize=(3, 5))
sns.boxplot(data = dfa, x = 'cluster_label', y = 'A0snap_category_performance', color = 'pink')
plt.title(title + ': A0snap_category_performance by group cluster')

In [ ]:
rp.summary_cont(dfa['A0snap_category_performance'].groupby(dfa['cluster_label']))

In [ ]:
grpe0 = dfa['A0snap_category_performance'][dfa['cluster_label']==0]
grpe1 = dfa['A0snap_category_performance'][dfa['cluster_label']==1]
grpe2 = dfa['A0snap_category_performance'][dfa['cluster_label']==2]

f_val, p_val = stats.f_oneway(grpe0, grpe1, grpe2)  
 
print( "ANOVA results: F=", f_val, ", P =", p_val) 

In [ ]:
stats.multicomp.pairwise_tukeyhsd

In [ ]:
# Tukey test
tukey = pairwise_tukeyhsd(endog=dfa['A0snap_category_performance'],
                          groups=dfa['cluster_label'],
                          alpha=0.05)

In [ ]:
print(tukey)

In [ ]:
dfa

In [ ]:
df_diff = dfa[dfa['cluster_label']!=1]

In [ ]:
title = df_diff.columns[0] + ' & ' + dfa.columns[1]

x ='A0snap_category_performance'
    #define mean and median
median = df_diff[x].median()
mean = "%.2f" % dfa[x].mean() 
    #plot figures
plt.figure(figsize=(3, 5))
sns.displot(data = df_diff, x = x, hue = 'cluster_label', palette = ['grey', 'yellow'])
plt.axvline(df_diff[x].median(), linestyle='dotted', color = 'magenta', label = f'median= {median}')
plt.axvline(df_diff[x].mean(), linestyle='dotted', color = 'blue', label = f'mean= {mean}')


plt.legend(loc = 'upper right')
plt.title(title + ': A0snap_category_performance by group cluster')